In [1]:
import pandas as pd
from datetime import date, timedelta, datetime
import numpy as np
from portfoliotools.screener.mutual_fund_screener import MutualFundScreener
import random
from tqdm import tqdm

In [2]:
def getFundsBelow52WHigh(sample = 50, drawdown_perc = 5, showAsDf = False):
    
    result  = pd.DataFrame()
    message = ""
    try:
        obj = MutualFundScreener()
        schemeCodes = obj.get_all_schemes()
        schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('Regular',na=False, case=False)]
        schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('idcw',na=False, case=False)]
        schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('dividend',na=False, case=False)]
        schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('capital withdrawal',na=False, case=False)]
        schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('bonus',na=False, case=False)]
        schemeCodes = list(schemeCodes.index.values)
        schemeCodes = random.sample(schemeCodes, sample)
        result = pd.DataFrame()

        for schemeCode in tqdm(schemeCodes):
            try:
                hist_nav = obj.get_historical_nav([schemeCode])
                if result.empty:
                    start_dt = hist_nav.index.min()
                    end_dt = hist_nav.index.max()
                    date_range = pd.date_range(start_dt, end_dt, freq='D')
                    hist_nav = hist_nav.reindex(index = date_range, method='ffill')
                    result = hist_nav
                else:
                    start_dt = min(result.index.min(),hist_nav.index.min())
                    end_dt = max(result.index.max(), hist_nav.index.max())
                    date_range = pd.date_range(start_dt, end_dt, freq='D')
                    result = result.reindex(index = date_range, method='ffill')
                    hist_nav = hist_nav.reindex(index = date_range, method='ffill')
                    result = result.join(hist_nav)
            except:
                pass
        drawdown = (1 - result/result.rolling(365).max())*100
        drawdown = drawdown.tail(1).transpose()
        drawdown = drawdown[drawdown[drawdown.columns.values[0]] > drawdown_perc]
        eligSchemes = drawdown.index.values
        schemes = obj.get_all_schemes()
        result = schemes[schemes.index.isin(eligSchemes)]

        for row in list(result.index.values):
            message += formatDrawDownMsg(result.loc[row])
    except:
        message = 'Failed to fetch fund details'
    
    return result if showAsDf else message

In [3]:
getFundsBelow52WHigh(10, 0, showAsDf=True)

100%|██████████| 10/10 [00:02<00:00,  3.80it/s]


,scheme_isin1,scheme_isin2,scheme_name
scheme_code,,,
101762,INF179K01608,-,HDFC Flexi Cap Fund - Growth Plan
141865,INF209KB1JB9,-,Aditya Birla Sun Life Resurgent India Fund - S...
147359,INF200KA1G34,-,SBI Capital Protection Oriented Fund - Series ...


In [4]:
obj = MutualFundScreener()
schemeCodes = obj.get_all_schemes()
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('Regular',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('idcw',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('dividend',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('capital withdrawal',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('bonus',na=False, case=False)]
schemeCodes = list(schemeCodes.index.values)
schemeCodes = random.sample(schemeCodes, 10)
result = pd.DataFrame()

for schemeCode in tqdm(schemeCodes):
    hist_nav = obj.get_historical_nav([schemeCode])
    if result.empty:
        start_dt = hist_nav.index.min()
        end_dt = hist_nav.index.max()
        date_range = pd.date_range(start_dt, end_dt, freq='D')
        hist_nav = hist_nav.reindex(index = date_range, method='ffill')
        result = hist_nav
    else:
        start_dt = min(result.index.min(),hist_nav.index.min())
        end_dt = max(result.index.max(), hist_nav.index.max())
        date_range = pd.date_range(start_dt, end_dt, freq='D')
        result = result.reindex(index = date_range, method='ffill')
        hist_nav = hist_nav.reindex(index = date_range, method='ffill')
        result = result.join(hist_nav)


100%|██████████| 10/10 [00:01<00:00,  6.31it/s]


In [5]:
drawdown = (1 - result/result.rolling(365).max())*100
drawdown = drawdown.tail(1).transpose()
drawdown = drawdown[drawdown[drawdown.columns.values[0]] > 5]
eligSchemes = drawdown.index.values
schemes = obj.get_all_schemes()
result = schemes[schemes.index.isin(eligSchemes)]

In [6]:
result

,scheme_isin1,scheme_isin2,scheme_name
scheme_code,,,


In [7]:
result

,scheme_isin1,scheme_isin2,scheme_name
scheme_code,,,


In [8]:
schemeCodes = obj.get_all_schemes()
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('Regular',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('idcw',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('dividend',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('capital withdrawal',na=False, case=False)]
schemeCodes = schemeCodes[~schemeCodes['scheme_name'].str.contains('bonus',na=False, case=False)]

In [9]:
debtFunds = schemeCodes[schemeCodes['scheme_name'].str.contains('Debt',na=False, case=False)]

In [10]:
schemeCodes.to_clipboard()